In [1]:
import numpy as np
import tensorflow as tf
import keras
import keras.backend as k

In [2]:
from keras.optimizers import SGD, Adam, Adagrad, RMSprop
from keras.applications import*
from keras.preprocessing import*
from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import load_img, img_to_array 
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation ,BatchNormalization, Dropout
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
import matplotlib.pyplot as plt

In [4]:
import glob
from PIL import Image
import os
from os import listdir

In [5]:
dirName = 'C:/Users/Admin/Desktop/Project/Digital Naturalist/Augmented data'
folders=listdir(dirName)
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles=list()
    for folders in listOfFile:
        fullPath=os.path.join(dirName,folders)
        allFiles.append(fullPath)
    return allFiles
Folders=getListOfFiles(dirName)
len(Folders)
subfolders=[]
for num in range(len(Folders)):
    sub_fols=getListOfFiles(Folders[num])
    subfolders+=sub_fols
subfolders

['C:/Users/Admin/Desktop/Project/Digital Naturalist/Augmented data\\Bird\\GIB_AUG',
 'C:/Users/Admin/Desktop/Project/Digital Naturalist/Augmented data\\Bird\\SPS_AUG',
 'C:/Users/Admin/Desktop/Project/Digital Naturalist/Augmented data\\Flower\\Corpse_AUG',
 'C:/Users/Admin/Desktop/Project/Digital Naturalist/Augmented data\\Flower\\LS_Orchid_AUG',
 'C:/Users/Admin/Desktop/Project/Digital Naturalist/Augmented data\\Mammal\\Pangolin_AUG',
 'C:/Users/Admin/Desktop/Project/Digital Naturalist/Augmented data\\Mammal\\SW_Deer_AUG']

In [6]:
X_data=[]
Y_data=[]
id_no=0
found=[]
for paths in subfolders:
    files = glob.glob(paths+"/*.jpg")
    found.append((paths.split('\\')[-2],paths.split('\\')[-1]))
    for myFile in files:
        img = Image.open(myFile)
        img = img.resize((224,224),Image.Resampling.LANCZOS)
        img = np.array(img)
        if img.shape==(224,224,3):
            X_data.append(img)
            Y_data.append(id_no)
    id_no+=1

In [7]:
X=np.array(X_data)
Y=np.array(Y_data)
print("X_shape",X.shape,"Y_shape",Y.shape)

X_shape (979, 224, 224, 3) Y_shape (979,)


In [8]:
X = X.astype('float32')/255.0
y_cat = to_categorical(Y_data,len(subfolders))
print(X.shape,y_cat.shape)
X_train,X_test,y_train,y_Test=train_test_split(X,y_cat,test_size=0.3)
print(str(len(X_train)))

(979, 224, 224, 3) (979, 6)
685


In [9]:
early_stop = EarlyStopping(monitor='val_accuracy', 
                           patience=5)

lr = ReduceLROnPlateau(monitor='val_accuaracy', 
                       factor=0.5, 
                       min_lr=0.01)

callback = [early_stop,lr]

In [10]:
def load_CNN():
    model=Sequential()
    model.add(Dropout(0.4,input_shape=(224,224,3)))
    model.add(Conv2D(256,(5,5),input_shape=(224,224,3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(128,(3,3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(6,activation='softmax'))
    return model
              
              

In [11]:
model = load_CNN()
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
weights = model.get_weights()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 220, 220, 256)     19456     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 256)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 128)     295040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        7

In [15]:
histories_acc=[]
histories_val_acc=[]
histories_loss=[]
histories_val_loss=[]
model.set_weights(weights)
h=model.fit(X_train,y_train,
            batch_size=6,
            verbose=2,
            callbacks=[early_stop],
            epochs=30,
            shuffle=True,
            validation_data=(X_test, y_Test))
model.summary()

Epoch 1/30
115/115 - 487s - loss: 2.0236 - accuracy: 0.1401 - val_loss: 1.7920 - val_accuracy: 0.1599 - 487s/epoch - 4s/step
Epoch 2/30
115/115 - 5204s - loss: 1.7934 - accuracy: 0.1460 - val_loss: 1.7914 - val_accuracy: 0.1599 - 5204s/epoch - 45s/step
Epoch 3/30


KeyboardInterrupt: 

In [13]:
model_json=model.to_json()
with open("final_model.json","w") as json_file:
    json_file.write(model_json)

In [14]:
model.save_weights("final_model.h5")
print("saved model to disk")

saved model to disk
